<a href="https://colab.research.google.com/github/BraedenTd/CMPS385SpringSemester/blob/main/Assignment_5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [64]:
!pip install pillow opencv-python numpy

!apt install fontconfig
!fc-list | grep "DejaVuSans.ttf"
!fc-list | grep "LiberationSerif-Regular.ttf"

!apt install ttf-mscorefonts-installer -y
!fc-cache -fv

import os
import cv2
import numpy as np
from PIL import Image, ImageDraw, ImageFont
import random
import string

# Letters to generate
letters = list(string.ascii_lowercase)

# Output folder
output_folder = "train_data"
os.makedirs(output_folder, exist_ok=True)

# Load fonts (modify path according to your system)
fonts = [
    "/usr/share/fonts/truetype/dejavu/DejaVuSans.ttf",
    "/usr/share/fonts/truetype/liberation/LiberationSerif-Regular.ttf",
]

# Check if fonts exist and install if necessary
for font_path in fonts:
    if not os.path.exists(font_path):
        print(f"Font not found: {font_path}")
        # Attempt to install the DejaVu font if not found
        if "DejaVuSans.ttf" in font_path:
            !apt-get install -y fonts-dejavu
        elif "LiberationSerif-Regular.ttf" in font_path:
            !apt-get install -y fonts-liberation
        else:
            raise FileNotFoundError(f"Font not found and cannot be automatically installed: {font_path}")

        # Check again if font exists after installation
        if not os.path.exists(font_path):
            raise FileNotFoundError(f"Font not found: {font_path}")
        else:
            print(f"Font {font_path} installed successfully.")

def add_noise(img):
    """Add random noise to the image."""
    noise = np.random.randint(0, 30, img.shape, dtype='uint8')
    noisy_img = cv2.add(img, noise)
    return noisy_img

def apply_blur(img):
    """Apply slight Gaussian blur to simulate ink spread."""
    return cv2.GaussianBlur(img, (3, 3), sigmaX=1)

def elastic_transform(image, alpha, sigma):
    """Apply elastic deformation to simulate realistic handwriting flow."""
    random_state = np.random.RandomState(None)
    shape = image.shape

    dx = (random_state.rand(*shape) * 2 - 1)
    dy = (random_state.rand(*shape) * 2 - 1)

    dx = cv2.GaussianBlur(dx, (15, 15), sigma) * alpha
    dy = cv2.GaussianBlur(dy, (15, 15), sigma) * alpha

    x, y = np.meshgrid(np.arange(shape[1]), np.arange(shape[0]))
    map_x = (x + dx).astype(np.float32)
    map_y = (y + dy).astype(np.float32)

    return cv2.remap(image, map_x, map_y, interpolation=cv2.INTER_LINEAR, borderMode=cv2.BORDER_REFLECT)

def generate_samples(letter, save_path, num_samples=5):
    os.makedirs(save_path, exist_ok=True)
    for i in range(num_samples):
        font_path = random.choice(fonts)
        font_size = random.randint(40, 60)
        font = ImageFont.truetype(font_path, font_size)

        img = Image.new('L', (50, 70), color=255)
        d = ImageDraw.Draw(img)
        d.text((5, 5), letter, font=font, fill=0)

        img = img.rotate(random.randint(-15, 15), expand=1, fillcolor=255)
        img = img.crop(img.getbbox())
        img = img.resize((30, 60))

        img_np = np.array(img)

        # Apply realistic distortions
        if random.random() < 0.5:
            img_np = elastic_transform(img_np, alpha=2, sigma=0.8)
        if random.random() < 0.5:
            img_np = apply_blur(img_np)
        if random.random() < 0.5:
            img_np = add_noise(img_np)

        cv2.imwrite(os.path.join(save_path, f"{letter}_{i}.png"), img_np)

def generate_test_image(word="abcdefghijklmnopqrstuvwxyz", filename="test_image.png"):
    font_path = random.choice(fonts)
    font = ImageFont.truetype(font_path, 50)

    img = Image.new('L', (len(word) * 35, 70), color=255)
    d = ImageDraw.Draw(img)
    for idx, letter in enumerate(word):
        d.text((idx * 35 + 5, 10), letter, font=font, fill=0)

    img_np = np.array(img)

    # Also apply transformations to the test image
    img_np = apply_blur(img_np)
    img_np = add_noise(img_np)

    cv2.imwrite(filename, img_np)

# Generate training samples
for letter in letters:
    save_path = os.path.join(output_folder, letter)
    generate_samples(letter, save_path)

# Generate a test image
generate_test_image()

print("✅ Realistic handwriting dataset generated! Check 'train_data/' and 'test_image.png'.")

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
fontconfig is already the newest version (2.13.1-4.2ubuntu5).
0 upgraded, 0 newly installed, 0 to remove and 34 not upgraded.
/usr/share/fonts/truetype/liberation/LiberationSerif-Regular.ttf: Liberation Serif:style=Regular
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
ttf-mscorefonts-installer is already the newest version (3.8ubuntu2).
0 upgraded, 0 newly installed, 0 to remove and 34 not upgraded.
/usr/share/fonts: caching, new cache contents: 0 fonts, 2 dirs
/usr/share/fonts/X11: caching, new cache contents: 0 fonts, 2 dirs
/usr/share/fonts/X11/encodings: caching, new cache contents: 0 fonts, 1 dirs
/usr/share/fonts/X11/encodings/large: caching, new cache contents: 0 fonts, 0 dirs
/usr/share/fonts/X11/util: caching, new cache contents: 0 fonts, 0 dirs
/usr/share/fonts/truetype: caching, new cache contents: 0 fonts, 3 dirs
/usr/share/font

In [65]:
import cv2
import numpy as np
import os
from hmmlearn import hmm
from skimage.feature import hog
from sklearn.preprocessing import StandardScaler

# -------------------------
# 1. Feature Extraction
# -------------------------
def preprocess_image(image_path):
    img = cv2.imread(image_path)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    _, thresh = cv2.threshold(gray, 128, 255, cv2.THRESH_BINARY_INV | cv2.THRESH_OTSU)
    return thresh

def extract_features(image):
    hog_features = hog(image, orientations=9, pixels_per_cell=(8, 8),
                       cells_per_block=(2, 2), visualize=False, feature_vector=True)
    return hog_features

# -------------------------
# 2. Training Phase
# -------------------------
def train_character_models(train_folder):
    models = {}
    scaler = StandardScaler()
    all_features = []

    # First pass: collect all features to normalize
    for label in os.listdir(train_folder):
        label_folder = os.path.join(train_folder, label)
        if not os.path.isdir(label_folder):
            continue
        for img_file in os.listdir(label_folder):
            img_path = os.path.join(label_folder, img_file)
            img = preprocess_image(img_path)
            feat = extract_features(img)
            all_features.append(feat)

    # Fit scaler
    scaler.fit(all_features)

    # Second pass: train HMMs
    for label in os.listdir(train_folder):
        label_folder = os.path.join(train_folder, label)
        if not os.path.isdir(label_folder):
            continue
        feats = []
        for img_file in os.listdir(label_folder):
            img_path = os.path.join(label_folder, img_file)
            img = preprocess_image(img_path)
            feat = extract_features(img)
            feats.append(scaler.transform([feat])[0])  # Normalize

        feats = np.array(feats)

        model = hmm.GaussianHMM(n_components=5, covariance_type='diag', n_iter=1000)
        model.fit(feats)
        models[label] = model
        print(f"Trained HMM for character: {label}")

    return models, scaler

# -------------------------
# 3. Prediction Phase
# -------------------------
def predict_text(models, scaler, test_image_path):
    image = preprocess_image(test_image_path)

    # Assume fixed-size character segmentation
    char_width = 30
    predicted_text = ""

    for x in range(0, image.shape[1], char_width):
        char_img = image[:, x:x+char_width]
        if char_img.shape[1] < char_width // 2:
            continue  # too small, skip

        char_img = cv2.resize(char_img, (30, 60))
        feature = extract_features(char_img)
        feature = scaler.transform([feature])[0]

        best_score = float('-inf')
        best_char = None

        for label, model in models.items():
            try:
                score = model.score([feature])
                if score > best_score:
                    best_score = score
                    best_char = label
            except:
                continue

        if best_char:
            predicted_text += best_char
        else:
            predicted_text += "?"

    return predicted_text

# -------------------------
# 4. Main function
# -------------------------
def main():
    train_folder = "train_data/"  # should contain folders 'a/', 'b/', 'c/', etc., with images
    test_image = "testimage.png"  # an image containing handwriting

    models, scaler = train_character_models(train_folder)
    result = predict_text(models, scaler, test_image)
    print("Predicted text:", result)

if __name__ == "__main__":
    main()


Trained HMM for character: o
Trained HMM for character: a
Trained HMM for character: y
Trained HMM for character: k


Trained HMM for character: u
Trained HMM for character: q
Trained HMM for character: x
Trained HMM for character: t
Trained HMM for character: b
Trained HMM for character: i
Trained HMM for character: c
Trained HMM for character: r


Trained HMM for character: s
Trained HMM for character: d
Trained HMM for character: m
Trained HMM for character: z
Trained HMM for character: v
Trained HMM for character: n
Trained HMM for character: h
Trained HMM for character: f
Trained HMM for character: e
Trained HMM for character: w
Trained HMM for character: l
Trained HMM for character: j
Trained HMM for character: g
Trained HMM for character: p
Predicted text: uluuuuuuuuuuuuuuuuwswwuuwsuusssussssswswsuuuuuuilmisisssskk
